In [46]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy
import time

hook = sy.TorchHook(torch)

In [2]:
# MovieLense DataSet Input

rs_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
train_data = pd.read_csv('Data/ua.base', sep='\t', names=rs_cols, encoding='latin-1')
test_data = pd.read_csv('Data/ua.test', sep='\t', names=rs_cols, encoding='latin-1')

In [3]:
#Training Data PreProcessing

user_ids = train_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = train_data["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
train_data["user"] = train_data["user_id"].map(user2user_encoded)
train_data["movie"] = train_data["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
train_data["rating"] = train_data["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings
min_rating = min(train_data["rating"])
max_rating = max(train_data["rating"])

print(
    "Training: Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Training: Number of users: 943, Number of Movies: 1680, Min rating: 1.0, Max rating: 5.0


In [4]:
#Testing Data PreProcessing

user_ids_test = test_data["user_id"].unique().tolist()
user2user_encoded_test = {x: i for i, x in enumerate(user_ids_test)}
userencoded2user_test = {i: x for i, x in enumerate(user_ids_test)}
movie_ids_test = test_data["movie_id"].unique().tolist()
movie2movie_encoded_test = {x: i for i, x in enumerate(movie_ids_test)}
movie_encoded2movie_test = {i: x for i, x in enumerate(movie_ids_test)}
test_data["user"] = test_data["user_id"].map(user2user_encoded_test)
test_data["movie"] = test_data["movie_id"].map(movie2movie_encoded_test)

num_users_test = len(user2user_encoded_test)
num_movies_test = len(movie_encoded2movie_test)
test_data["rating"] = test_data["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings
min_rating_test = min(test_data["rating"])
max_rating_test = max(test_data["rating"])

print(
    "Testing: Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users_test, num_movies_test, min_rating_test, max_rating_test
    )
)

Testing: Number of users: 943, Number of Movies: 1129, Min rating: 1.0, Max rating: 5.0


In [5]:
train_data

,user_id,movie_id,rating,unix_timestamp,user,movie
0,1,1,5.0,874965758,0,0
1,1,2,3.0,876893171,0,1
2,1,3,4.0,878542960,0,2
3,1,4,3.0,876893119,0,3
4,1,5,3.0,889751712,0,4
...,...,...,...,...,...,...
90565,943,1047,2.0,875502146,942,1034
90566,943,1074,4.0,888640250,942,1063
90567,943,1188,3.0,888640250,942,1173
90568,943,1228,3.0,888640275,942,1211


In [6]:
train_input = train_data[["user", "movie"]].values
# Normalize the ratings between 0 and 1
train_output = train_data["rating"].apply(lambda x: (x - min_rating)/(max_rating - min_rating)).values


test_input = test_data[["user", "movie"]].values
# Normalize the ratings between 0 and 1
test_output = test_data["rating"].apply(lambda x: (x - min_rating_test)/(max_rating_test - min_rating_test)).values

In [9]:
train_output

array([1.  , 0.5 , 0.75, ..., 0.5 , 0.5 , 0.5 ])

In [15]:
train_input

array([[   0,    0],
       [   0,    1],
       [   0,    2],
       ...,
       [ 942, 1173],
       [ 942, 1211],
       [ 942, 1322]])

In [37]:
#Create virtual workers

workers_nodes = []

#training
workers_train_data_pointers = []

for user_id in user_ids:
    worker = sy.VirtualWorker(hook, id="user_"+str(user_id))
    data_pointer = sy.BaseDataset(torch.tensor(train_input), torch.tensor(train_output)).send(worker)
    workers_nodes.append(worker)
    workers_train_data_pointers.append(data_pointer)

    
#testing
workers_test_data_pointers = []

for user_id in user_ids:
    data_pointer = sy.BaseDataset(torch.tensor(test_input), torch.tensor(test_output)).send(workers_nodes[user_id-1])
    workers_test_data_pointers.append(data_pointer)

In [38]:
BATCH_SIZE=64

# Create federated training datasets
federated_train_dataset = sy.FederatedDataset(workers_train_data_pointers)
# Create federated training dataloaders
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size=BATCH_SIZE)

# Create federated testing datasets
federated_test_dataset = sy.FederatedDataset(workers_test_data_pointers)
# Create federated testing dataloaders
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, shuffle=True, batch_size=BATCH_SIZE)

In [42]:
#Create model

EMBEDDING_SIZE = 50

class Model(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size):
        super(Model, self).__init__()
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        
        self.fc_layers = nn.ModuleList()
        self.fc_layers.append(nn.Linear(50, 50))
        self.output_layer = nn.Linear(50, 1)

    def forward(self, train_data):
        users = torch.as_tensor(train_data["user"])
        movies = torch.as_tensor(train_data["movie"])
        user_embedding_x = user_embedding(users)
        movie_embedding_y = movie_embedding(movies)
        # concatenate user and movie embeddings to form input to 1 dim
        x = torch.cat([user_embedding_x, movie_embedding_y], 1)
        
        for idx, _ in enumerate(range(len(self.fc_layers))):
            x = self.fc_layers[idx](x)
            x = F.dropout(x, p=0.2)
            x = F.batch_norm(x)
            x = F.relu(x)
        
        logit = self.output_layer(x)
        # The sigmoid activation forces the rating to between 0 and 1
        rating = torch.sigmoid(logit)
        return rating

model = Model(num_users, num_movies, EMBEDDING_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
print(model)

Model(
  (user_embedding): Embedding(943, 50)
  (movie_embedding): Embedding(1680, 50)
  (fc_layers): ModuleList(
    (0): Linear(in_features=50, out_features=50, bias=True)
  )
  (output_layer): Linear(in_features=50, out_features=1, bias=True)
)


In [50]:
#Train model

def train():
    for epoch in range(0, 5):
        for batch_idx, (data, target) in enumerate(federated_train_loader):
            # send the model to the client device where the data is present
            #model.send(data.location)
            # training the model
            optimizer.zero_grad()
            print(data)
#             prediction = model(data)
#             loss = F.mse_loss(prediction.view(-1), target)
#             print(loss)
#             loss.backward()
#             optimizer.step()
#             # get back the improved model
#             model.get()
#             return utils.federated_avg({
#                 "model": model
#             })


In [51]:

def test(federated_model):
    federated_model.eval()
    test_loss = 0
    for data, target in federated_test_loader:
        output = federated_model(data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item()
        predection = output.data.max(1, keepdim=True)[1]
        
    test_loss /= len(federated_test_loader.dataset)
    print('Test set: Average loss: {:.4f}'.format(test_loss))

In [52]:
for epoch in range(5):
    start_time = time.time()
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    model = federated_model
    test(federated_model)
    total_time = time.time() - start_time
    print('Communication time over the network', round(total_time, 2), 's\n')

Epoch Number 1
(Wrapper)>[PointerTensor | me:90655228876 -> user_1:69222299275]
(Wrapper)>[PointerTensor | me:57423738144 -> user_1:371897684]
(Wrapper)>[PointerTensor | me:95689145541 -> user_1:60260877666]
(Wrapper)>[PointerTensor | me:89372591247 -> user_1:86376483020]
(Wrapper)>[PointerTensor | me:81352581933 -> user_1:93729034951]
(Wrapper)>[PointerTensor | me:93515142250 -> user_1:88719789188]
(Wrapper)>[PointerTensor | me:38832578806 -> user_1:35178522758]
(Wrapper)>[PointerTensor | me:32142760305 -> user_1:78948692323]
(Wrapper)>[PointerTensor | me:71280796974 -> user_1:37994850156]
(Wrapper)>[PointerTensor | me:29242258017 -> user_1:93349903511]
(Wrapper)>[PointerTensor | me:76446657602 -> user_1:63966301256]
(Wrapper)>[PointerTensor | me:57182551600 -> user_1:63815136446]
(Wrapper)>[PointerTensor | me:65051604524 -> user_1:18721574490]
(Wrapper)>[PointerTensor | me:36171422309 -> user_1:38687080500]
(Wrapper)>[PointerTensor | me:19763618782 -> user_1:38678241289]
(Wrapper)>[P

(Wrapper)>[PointerTensor | me:72576307582 -> user_1:83607060425]
(Wrapper)>[PointerTensor | me:8007219090 -> user_1:39655838592]
(Wrapper)>[PointerTensor | me:70719852567 -> user_1:96509686773]
(Wrapper)>[PointerTensor | me:76350186918 -> user_1:16610303531]
(Wrapper)>[PointerTensor | me:55432210018 -> user_1:34526430452]
(Wrapper)>[PointerTensor | me:40895725894 -> user_1:63737102566]
(Wrapper)>[PointerTensor | me:93118322752 -> user_1:1246518731]
(Wrapper)>[PointerTensor | me:30446738882 -> user_1:22204267626]
(Wrapper)>[PointerTensor | me:23135056138 -> user_1:62336084801]
(Wrapper)>[PointerTensor | me:78737698523 -> user_1:2698015501]
(Wrapper)>[PointerTensor | me:71427801517 -> user_1:99791073324]
(Wrapper)>[PointerTensor | me:27402159768 -> user_1:3254680668]
(Wrapper)>[PointerTensor | me:34978608635 -> user_1:59922058164]
(Wrapper)>[PointerTensor | me:99953559024 -> user_1:61105404661]
(Wrapper)>[PointerTensor | me:61506970090 -> user_1:36756023943]
(Wrapper)>[PointerTensor | me

(Wrapper)>[PointerTensor | me:24506219991 -> user_1:39842315497]
(Wrapper)>[PointerTensor | me:79840507006 -> user_1:9539904653]
(Wrapper)>[PointerTensor | me:88000231183 -> user_1:62668992666]
(Wrapper)>[PointerTensor | me:46409350367 -> user_1:99542366103]
(Wrapper)>[PointerTensor | me:62882528599 -> user_1:16476619785]
(Wrapper)>[PointerTensor | me:55635386397 -> user_1:76170096877]
(Wrapper)>[PointerTensor | me:10958053927 -> user_1:80634404648]
(Wrapper)>[PointerTensor | me:22182574171 -> user_1:61704325760]
(Wrapper)>[PointerTensor | me:59726209039 -> user_1:9770997462]
(Wrapper)>[PointerTensor | me:80002689827 -> user_1:54000047381]
(Wrapper)>[PointerTensor | me:25948660573 -> user_1:5543487569]
(Wrapper)>[PointerTensor | me:67467868812 -> user_1:22045979614]
(Wrapper)>[PointerTensor | me:33920842552 -> user_1:89034403792]
(Wrapper)>[PointerTensor | me:91905856079 -> user_1:74746212990]
(Wrapper)>[PointerTensor | me:6162497371 -> user_1:57922409861]
(Wrapper)>[PointerTensor | me

(Wrapper)>[PointerTensor | me:49017480025 -> user_1:84249267523]
(Wrapper)>[PointerTensor | me:71300459905 -> user_1:327409410]
(Wrapper)>[PointerTensor | me:93710376664 -> user_1:24469869101]
(Wrapper)>[PointerTensor | me:6393048715 -> user_1:71814611396]
(Wrapper)>[PointerTensor | me:53625106978 -> user_1:50099078634]
(Wrapper)>[PointerTensor | me:43250342971 -> user_1:45775342151]
(Wrapper)>[PointerTensor | me:10339671636 -> user_1:19982352865]
(Wrapper)>[PointerTensor | me:78849868423 -> user_1:29939909241]
(Wrapper)>[PointerTensor | me:5531925550 -> user_1:45843205417]
(Wrapper)>[PointerTensor | me:89032530878 -> user_1:50170865803]
(Wrapper)>[PointerTensor | me:31852610836 -> user_1:35618220334]
(Wrapper)>[PointerTensor | me:40430412681 -> user_1:77718626931]
(Wrapper)>[PointerTensor | me:61930919012 -> user_1:9703086077]
(Wrapper)>[PointerTensor | me:78413800177 -> user_1:84265718427]
(Wrapper)>[PointerTensor | me:82411397358 -> user_1:17191266341]
(Wrapper)>[PointerTensor | me:

(Wrapper)>[PointerTensor | me:76332535216 -> user_1:69234358508]
(Wrapper)>[PointerTensor | me:33226858289 -> user_1:90437942613]
(Wrapper)>[PointerTensor | me:52113481847 -> user_1:62118249823]
(Wrapper)>[PointerTensor | me:60205997109 -> user_1:81662779838]
(Wrapper)>[PointerTensor | me:68173648585 -> user_1:37552133145]
(Wrapper)>[PointerTensor | me:64061393281 -> user_1:20714047382]
(Wrapper)>[PointerTensor | me:11139164605 -> user_1:52373459125]
(Wrapper)>[PointerTensor | me:62569450492 -> user_1:78884801558]
(Wrapper)>[PointerTensor | me:98825071723 -> user_1:75981005212]
(Wrapper)>[PointerTensor | me:1456129478 -> user_1:22693546884]
(Wrapper)>[PointerTensor | me:52281647269 -> user_1:81921754570]
(Wrapper)>[PointerTensor | me:83185318013 -> user_1:98048089306]
(Wrapper)>[PointerTensor | me:63888551528 -> user_1:36825310594]
(Wrapper)>[PointerTensor | me:87230972584 -> user_1:85017975849]
(Wrapper)>[PointerTensor | me:83869761816 -> user_1:49332139909]
(Wrapper)>[PointerTensor |

...

(Wrapper)>[PointerTensor | me:10789885096 -> user_2:12650186757]
(Wrapper)>[PointerTensor | me:83577862822 -> user_2:27891932931]
(Wrapper)>[PointerTensor | me:60470747063 -> user_2:75303242846]
(Wrapper)>[PointerTensor | me:54113312013 -> user_2:60566160475]
(Wrapper)>[PointerTensor | me:2287555611 -> user_2:68708009330]
(Wrapper)>[PointerTensor | me:35925360977 -> user_2:15762315679]
(Wrapper)>[PointerTensor | me:59759018589 -> user_2:43913346960]
(Wrapper)>[PointerTensor | me:44087417998 -> user_2:94914608121]
(Wrapper)>[PointerTensor | me:21622638355 -> user_2:65897001004]
(Wrapper)>[PointerTensor | me:82572717632 -> user_2:47579624609]
(Wrapper)>[PointerTensor | me:220492700 -> user_2:19595596168]
(Wrapper)>[PointerTensor | me:73195073779 -> user_2:20789460188]
(Wrapper)>[PointerTensor | me:23470591514 -> user_2:56725000547]
(Wrapper)>[PointerTensor | me:49656555061 -> user_2:21652872355]
(Wrapper)>[PointerTensor | me:73566088225 -> user_2:57818533003]
(Wrapper)>[PointerTensor | m

(Wrapper)>[PointerTensor | me:20242017085 -> user_2:97903435019]
(Wrapper)>[PointerTensor | me:70232311265 -> user_2:18641555722]
(Wrapper)>[PointerTensor | me:14157252490 -> user_2:29737482144]
(Wrapper)>[PointerTensor | me:69054457700 -> user_2:41809175393]
(Wrapper)>[PointerTensor | me:53986349 -> user_2:79192775245]
(Wrapper)>[PointerTensor | me:99673114529 -> user_2:69620651273]
(Wrapper)>[PointerTensor | me:25695403907 -> user_2:89040566474]
(Wrapper)>[PointerTensor | me:82172193634 -> user_2:39148952971]
(Wrapper)>[PointerTensor | me:45301023224 -> user_2:98043715795]
(Wrapper)>[PointerTensor | me:75750717709 -> user_2:41424739169]
(Wrapper)>[PointerTensor | me:69718725757 -> user_2:29677685722]
(Wrapper)>[PointerTensor | me:58491243059 -> user_2:68135702178]
(Wrapper)>[PointerTensor | me:84428237754 -> user_2:80087502597]
(Wrapper)>[PointerTensor | me:5480756234 -> user_2:5158656692]
(Wrapper)>[PointerTensor | me:33998484583 -> user_2:73195274777]
(Wrapper)>[PointerTensor | me:

(Wrapper)>[PointerTensor | me:50894494985 -> user_2:87694317808]
(Wrapper)>[PointerTensor | me:73031249117 -> user_2:92125166884]
(Wrapper)>[PointerTensor | me:88557902820 -> user_2:19923703167]
(Wrapper)>[PointerTensor | me:50024495433 -> user_2:67008200594]
(Wrapper)>[PointerTensor | me:40379838006 -> user_2:68825405478]
(Wrapper)>[PointerTensor | me:46627565789 -> user_2:42080614548]
(Wrapper)>[PointerTensor | me:7954347840 -> user_2:85908734387]
(Wrapper)>[PointerTensor | me:11339806810 -> user_2:12774293432]
(Wrapper)>[PointerTensor | me:99653495634 -> user_2:45068034692]
(Wrapper)>[PointerTensor | me:38354050407 -> user_2:33973320718]
(Wrapper)>[PointerTensor | me:57418137859 -> user_2:23188166898]
(Wrapper)>[PointerTensor | me:24514242973 -> user_2:48454297004]
(Wrapper)>[PointerTensor | me:95521717042 -> user_2:49390131316]
(Wrapper)>[PointerTensor | me:73555395277 -> user_2:28824623392]
(Wrapper)>[PointerTensor | me:58757938373 -> user_2:83338850323]
(Wrapper)>[PointerTensor |

(Wrapper)>[PointerTensor | me:34154227818 -> user_2:3581895137]
(Wrapper)>[PointerTensor | me:12544703786 -> user_2:90733555464]
(Wrapper)>[PointerTensor | me:35849515020 -> user_2:68820217134]
(Wrapper)>[PointerTensor | me:5176335613 -> user_2:92834601590]
(Wrapper)>[PointerTensor | me:12736519609 -> user_2:73136595982]
(Wrapper)>[PointerTensor | me:58182449241 -> user_2:28833304599]
(Wrapper)>[PointerTensor | me:45545094868 -> user_2:49389817624]
(Wrapper)>[PointerTensor | me:23212684460 -> user_2:97779480263]
(Wrapper)>[PointerTensor | me:60008761256 -> user_2:24301425780]
(Wrapper)>[PointerTensor | me:72552691752 -> user_2:43526986465]
(Wrapper)>[PointerTensor | me:81492628034 -> user_2:56677203854]
(Wrapper)>[PointerTensor | me:62897964246 -> user_2:17469564491]
(Wrapper)>[PointerTensor | me:11459786296 -> user_2:10630116825]
(Wrapper)>[PointerTensor | me:10006848847 -> user_2:87779585455]
(Wrapper)>[PointerTensor | me:3980941810 -> user_2:26051227870]
(Wrapper)>[PointerTensor | m

KeyboardInterrupt: 